### Parse source data to parquet

Note: All files has been parsed and upload to the Intermediate_Files Folder

In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

### Read in raw data

In [2]:
# as raw data file is large for github, download data to local
# adjust the path to your local path
path = "../00_Source_Data/arcos_all_washpost.tsv"
csv_chunk = pd.read_table(path, chunksize=50_000, low_memory=False)

### Washington

In [5]:
Washington = []
for i, chunk in enumerate(csv_chunk):
    append_chunk = chunk.loc[chunk["BUYER_STATE"] == "WA"]
    Washington.append(append_chunk)
data_selected_Washington = pd.concat(Washington)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
data_selected_Washington["NDC_NO"] = data_selected_Washington["NDC_NO"].astype(str)

In [ ]:
data_selected_Washington.sample(20)

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DRUG_NAME,Measure,MME_Conversion_Factor,Dosage_Strength,TRANSACTION_DATE,Combined_Labeler_Name,Reporter_family,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
69529076,RW0191813,DISTRIBUTOR,CARDINAL HEALTH,NaN,"801 C STREET NW, SUITE B",NaN,AUBURN,WA,98001,KING,...,OXYCODONE,TAB,1.5,5.0,2016-12-21,Amneal,Cardinal Health,1.793000,400.0,2689.5000
75178886,RW0191813,DISTRIBUTOR,CARDINAL HEALTH,NaN,"801 C STREET NW, SUITE B",NaN,AUBURN,WA,98001,KING,...,OXYCODONE,TAB,1.5,7.5,2017-10-05,"Alvogen, Inc.",Cardinal Health,1.344750,200.0,2017.1250
61006458,RT0424820,DISTRIBUTOR,THRIFTY WHITE WAREHOUSE #899,NaN,6055 NATHAN LN N STE 200B,NaN,PLYMOUTH,MN,55442,HENNEPIN,...,OXYCODONE,TAB,1.5,10.0,2016-04-06,Teva,Thrifty White Warehouse,0.896500,100.0,1344.7500
116799645,RW0191813,DISTRIBUTOR,CARDINAL HEALTH,NaN,"801 C STREET NW, SUITE B",NaN,AUBURN,WA,98001,KING,...,OXYCODONE,TAB,1.5,10.0,2009-06-26,SpecGx LLC,Cardinal Health,1.793000,200.0,2689.5000
69617524,RA0481894,DISTRIBUTOR,"ANDA PHARMACEUTICALS, INC",NaN,8644 POLK LN,NaN,OLIVE BRANCH,MS,38654,DESOTO,...,HYDROCODONE,TAB,1.0,7.5,2017-12-12,SpecGx LLC,"Anda, Inc",0.908100,200.0,908.1000
61175802,PM0022929,DISTRIBUTOR,MCKESSON CORPORATION,DBA MCKESSON DRUG CO.,9700 SW COMMERCE CIRCLE,NaN,WILSONVILLE,OR,97070,CLACKAMAS,...,HYDROCODONE,TAB,1.0,10.0,2018-04-09,SpecGx LLC,McKesson Corporation,21.794400,3600.0,21794.4000
275555343,RW0204026,DISTRIBUTOR,WALGREEN CO,NaN,DBA: WALGREEN WAREHOUSE,2370 E MAIN STREET,WOODLAND,CA,95776,YOLO,...,OXYCODONE,TAB,1.5,80.0,2008-07-07,Purdue,Walgreen Co,14.344000,200.0,21516.0000
279361510,RW0191813,DISTRIBUTOR,CARDINAL HEALTH,NaN,"801 C STREET NW, SUITE B",NaN,AUBURN,WA,98001,KING,...,OXYCODONE,TAB,1.5,7.5,2014-08-25,SpecGx LLC,Cardinal Health,0.672375,100.0,1008.5625
279153734,RW0204026,DISTRIBUTOR,WALGREEN CO,NaN,DBA: WALGREEN WAREHOUSE,2370 E MAIN STREET,WOODLAND,CA,95776,YOLO,...,OXYCODONE,TAB,1.5,5.0,2013-07-15,Actavis,Walgreen Co,2.241250,500.0,3361.8750
67149668,RW0191813,DISTRIBUTOR,CARDINAL HEALTH,NaN,"801 C STREET NW, SUITE B",NaN,AUBURN,WA,98001,KING,...,HYDROCODONE,TAB,1.0,5.0,2017-06-29,"Lupin Pharmaceuticals,Inc.",Cardinal Health,0.302700,100.0,302.7000


In [ ]:
data_selected_Washington["DRUG_NAME"].value_counts()

DRUG_NAME
OXYCODONE      4195239
HYDROCODONE    3624513
Name: count, dtype: int64

In [ ]:
data_selected_Washington["MME"].value_counts()

MME
1513.500      545588
2689.500      537058
1344.750      521849
4034.250      490006
3027.000      445896
               ...  
82029.750          1
2361.060           1
161278.560         1
1589.175           1
310637.250         1
Name: count, Length: 1097, dtype: int64

In [ ]:
data_selected_Washington["BUYER_STATE"].value_counts()

BUYER_STATE
WA    7819752
Name: count, dtype: int64

In [ ]:
# data_selected_Washington.to_parquet(".../20_Intermediate_Files/Washington.parquet")
# not run this code if the file already there
# uncomment if want to create/update the file

### Texas

In [ ]:
# as raw data file is large for github, download data to local
# adjust the path to your local path
path = "../Data/arcos_all_washpost.tsv"
csv_chunk = pd.read_table(path, chunksize=50_000, low_memory=False)

In [ ]:
Texas = []
for i, chunk in enumerate(csv_chunk):
    append_chunk = chunk.loc[chunk["BUYER_STATE"] == "TX"]
    Texas.append(append_chunk)
data_selected_Texas = pd.concat(Texas)

In [ ]:
data_selected_Texas["NDC_NO"] = data_selected_Texas["NDC_NO"].astype(str)

In [ ]:
data_selected_Texas.sample(20)

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DRUG_NAME,Measure,MME_Conversion_Factor,Dosage_Strength,TRANSACTION_DATE,Combined_Labeler_Name,Reporter_family,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
117975820,RW0279996,DISTRIBUTOR,CARDINAL HEALTH,NaN,851 HENRIETTA CREEK ROAD,NaN,ROANOKE,TX,76262,DENTON,...,HYDROCODONE,TAB,1.0,7.5,2015-01-20,"Par Pharmaceutical, Inc.",Cardinal Health,2.27025,500.0,2270.25
196542357,PW0186785,DISTRIBUTOR,WAL-MART PHARM WAREHOUSE #1,NaN,2252 NORTH 8TH STREET,NaN,ROGERS,AR,72756,BENTON,...,HYDROCODONE,TAB,1.0,10.0,2007-01-11,SpecGx LLC,Wal-Mart,1.21080,200.0,1210.80
205668999,RW0279996,DISTRIBUTOR,CARDINAL HEALTH,NaN,851 HENRIETTA CREEK ROAD,NaN,ROANOKE,TX,76262,DENTON,...,HYDROCODONE,TAB,1.0,7.5,2012-05-25,"UCB, Inc.",Cardinal Health,0.45405,100.0,454.05
207777061,PW0186785,DISTRIBUTOR,WAL-MART PHARM WAREHOUSE #1,NaN,2252 NORTH 8TH STREET,NaN,ROGERS,AR,72756,BENTON,...,HYDROCODONE,TAB,1.0,7.5,2008-02-19,SpecGx LLC,Wal-Mart,0.90810,200.0,908.10
220612138,RW0281953,DISTRIBUTOR,WALGREEN CO,NaN,710 OVILLA ROAD,NaN,WAXAHACHIE,TX,75167,ELLIS,...,HYDROCODONE,TAB,1.0,5.0,2006-02-10,SpecGx LLC,Walgreen Co,4.54050,1500.0,4540.50
274838026,PW0186785,DISTRIBUTOR,WAL-MART PHARM WAREHOUSE #1,NaN,2252 NORTH 8TH STREET,NaN,ROGERS,AR,72756,BENTON,...,HYDROCODONE,TAB,1.0,7.5,2011-03-04,SpecGx LLC,Wal-Mart,0.90810,200.0,908.10
222147055,RM0138328,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,4012 S PURDUE AVE,OKLAHOMA CITY,OK,73179,OKLAHOMA,...,HYDROCODONE,TAB,1.0,7.5,2010-04-29,SpecGx LLC,McKesson Corporation,0.45405,100.0,454.05
240405388,RC0275467,DISTRIBUTOR,"CVS PHARMACY, INC",NaN,800 CVS DRIVE,NaN,ENNIS,TX,75119,ELLIS,...,HYDROCODONE,TAB,1.0,7.5,2009-03-13,Actavis,CVS,2.27025,500.0,2270.25
220824032,RM0138328,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,4012 S PURDUE AVE,OKLAHOMA CITY,OK,73179,OKLAHOMA,...,OXYCODONE,TAB,1.5,10.0,2013-12-13,Amneal,McKesson Corporation,0.89650,100.0,1344.75
214479266,RM0328408,DISTRIBUTOR,MCKESSON CORPORATION,NaN,3301 POLLOK DRIVE,NaN,CONROE,TX,77303,MONTGOMERY,...,HYDROCODONE,TAB,1.0,10.0,2011-09-23,SpecGx LLC,McKesson Corporation,1.21080,200.0,1210.80


In [ ]:
# data_selected_Texas.to_parquet("../20_Intermediate_Files/Texas.parquet")
# not run this code if the file already there
# uncomment if want to create/update the file

### Florida

In [ ]:
# as raw data file is large for github, download data to local
# adjust the path to your local path
path = "../Data/arcos_all_washpost.tsv"
csv_chunk = pd.read_table(path, chunksize=50_000, low_memory=False)

In [ ]:
Florida = []
for i, chunk in enumerate(csv_chunk):
    append_chunk = chunk.loc[chunk["BUYER_STATE"] == "FL"]
    Florida.append(append_chunk)
data_selected_Florida = pd.concat(Florida)

In [ ]:
data_selected_Florida["NDC_NO"] = data_selected_Florida["NDC_NO"].astype(str)

In [ ]:
# data_selected_Florida.to_parquet("../20_Intermediate_Files/Florida.parquet")
# not run this code if the file already there
# uncomment if want to create/update the file